In [1]:
import pandas as pd

df = pd.read_csv("data/train.csv")
item_metadata_df = pd.read_csv("data/item_metadata.csv")
other = pd.read_json("data/id_mappings.json")

# Extract the dictionary inside "user_mapping"
user_mapping = other["user_mapping"]
item_mapping = other["item_mapping"]

# Convert to DataFrame
user_mapping_df = pd.DataFrame(list(user_mapping.items()), columns=["user_id", "user_index"])
item_mapping_df = pd.DataFrame(list(item_mapping.items()), columns=["item_str_code", "item_id"])

item_metadata_df["item_id"] = item_metadata_df["parent_asin"].map(item_mapping)
item_merge_df = pd.merge(item_metadata_df, item_mapping_df, how="left", left_on="parent_asin", right_on="item_str_code")
item_merge_df["item_id"] = item_merge_df["item_id_x"]



item_merge_df["item_id"] = item_merge_df["item_id"].astype(float)
item_merge_df = item_merge_df.drop('item_id_x', axis=1)
item_merge_df = item_merge_df.drop('item_id_y', axis=1)
item_merge_df = item_merge_df.drop('item_str_code', axis=1)


result = pd.merge(df, item_merge_df, how="left", left_on="item_id", right_on="item_id")
result

,user_id,item_id,rating,timestamp,parent_asin,main_category,title,average_rating,rating_number,price,store,features,description,images,categories,image_count,has_images,image_urls,category
0,0,7314,5.0,1353612262000,B006CQ8TC2,Appstore for Android,Turkey Season,3.5,304.0,0.99,"Gibbed Games, LLC","['Hunt and slay deviant turkeys, then go back ...",['Gobble gobble gobble! Turkeys are running am...,"{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/71oAt4H2...,Software
1,0,15493,5.0,1370653034000,B00B7S5FDG,Appstore for Android,Another Monster at the End of This Book...Star...,4.3,460.0,3.99,Sesame Workshop,['Bursting with innovative activities woven ri...,"[""Available for Kindle Fire tablets AND Fire T...","{'hi_res': array([None, None, None, None, None...",[],12,True,['https://m.media-amazon.com/images/I/B1dGfKbS...,Software
2,0,18817,4.0,1373668644000,B00DRPVE3M,Appstore for Android,UNO ™ & Friends - The Classic Card Game Goes S...,3.9,11842.0,NaN,Gameloft,['Create games and invite your friends to play...,"['UNO™ & Friends: Fast Fun for Everyone!UNO™, ...","{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/A1dc9cyK...,Software
3,0,18251,4.0,1373669469000,B00D948SS0,Appstore for Android,World's Biggest Wordsearch - Your daily free w...,3.8,8290.0,0.00,Supersonic Software Ltd,"['5,000 words to find' '361 puzzles based on a...","[""Find thousands of words in the biggest and b...","{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/A1cI1uy6...,Software
4,0,16199,1.0,1375136041000,B00BQPEIT2,Appstore for Android,Temple Run: Oz,4.0,1043.0,NaN,Disney,['Stunning environments inspired by the film –...,['The most thrilling running experience now co...,"{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/91BmBng4...,Software
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543141,868216,15203,5.0,1397341480000,B00B1TGUMG,Software,Microsoft Office Home and Student 2013 (1PC/1U...,4.3,1759.0,NaN,Microsoft,['NOTE: Windows XP and Vista NOT supported. Th...,['Product Description'\n 'Office Home & Studen...,{'hi_res': array(['https://m.media-amazon.com/...,['Software' 'Microsoft' 'All Microsoft'],6,True,['https://m.media-amazon.com/images/I/61vi4XLD...,Software
2543142,868216,51839,5.0,1500036628141,B06XS4YRWX,Software,"NTI Echo 3, Make an exact copy of a HDD or SSD...",3.5,17.0,NaN,Nti,"['Versatile: One-to-One Cloning, Dynamic Resiz...","['NTI Echo'\n ""is the perfect solution for all...","{'hi_res': array([None, None, None, None, None...",['Software' 'Utilities' 'Backup'],14,True,['https://m.media-amazon.com/images/I/81FlCZVs...,Software
2543143,868217,14052,4.0,1464923235000,B00AB7HESI,Appstore for Android,Scribblenauts Remix,3.8,5520.0,0.99,Warner Bros,['CLASSIC GAMEPLAY - 50 levels to play!'\n 'PL...,['THINK IT! CREATE IT! SOLVE IT! The award-win...,"{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/81bAX50X...,Software
2543144,868217,40099,4.0,1465146950000,B014RGFC0K,Appstore for Android,Smashy Road: Wanted,3.6,7541.0,0.00,Bearbit Studios B.V.,['90 unlockable vehicles!!'\n 'Random generate...,['You are WANTED! Take your driving to new hei...,"{'hi_res': array([None, None, None, None, None...",[],9,True,['https://m.media-amazon.com/images/I/71TPOiUY...,Software


In [2]:
# df_user = result[result["user_id"] == 0]
df_user = result
def normalize(x):
    return x/5

df_user.loc[:, "rating"] = df_user["rating"].apply(normalize)
df_user.loc[:, "average_rating"] = df_user["average_rating"].apply(normalize)  
df_user.loc[:, "average_rating"] = df_user["average_rating"]  * df_user["rating_number"] * df_user["rating"]
df_user = df_user.drop(["timestamp", "features", "image_urls", "image_count", "has_images", "categories", "images"], axis=1)
df_user0 = df_user[df_user["user_id"] == 0]
# df_user = df_user.drop(["item_id", "user_id", "parent_asin"], axis=1)
df_user

,user_id,item_id,rating,parent_asin,main_category,title,average_rating,rating_number,price,store,description,category
0,0,7314,1.0,B006CQ8TC2,Appstore for Android,Turkey Season,212.800,304.0,0.99,"Gibbed Games, LLC",['Gobble gobble gobble! Turkeys are running am...,Software
1,0,15493,1.0,B00B7S5FDG,Appstore for Android,Another Monster at the End of This Book...Star...,395.600,460.0,3.99,Sesame Workshop,"[""Available for Kindle Fire tablets AND Fire T...",Software
2,0,18817,0.8,B00DRPVE3M,Appstore for Android,UNO ™ & Friends - The Classic Card Game Goes S...,7389.408,11842.0,NaN,Gameloft,"['UNO™ & Friends: Fast Fun for Everyone!UNO™, ...",Software
3,0,18251,0.8,B00D948SS0,Appstore for Android,World's Biggest Wordsearch - Your daily free w...,5040.320,8290.0,0.00,Supersonic Software Ltd,"[""Find thousands of words in the biggest and b...",Software
4,0,16199,0.2,B00BQPEIT2,Appstore for Android,Temple Run: Oz,166.880,1043.0,NaN,Disney,['The most thrilling running experience now co...,Software
...,...,...,...,...,...,...,...,...,...,...,...,...
2543141,868216,15203,1.0,B00B1TGUMG,Software,Microsoft Office Home and Student 2013 (1PC/1U...,1512.740,1759.0,NaN,Microsoft,['Product Description'\n 'Office Home & Studen...,Software
2543142,868216,51839,1.0,B06XS4YRWX,Software,"NTI Echo 3, Make an exact copy of a HDD or SSD...",11.900,17.0,NaN,Nti,"['NTI Echo'\n ""is the perfect solution for all...",Software
2543143,868217,14052,0.8,B00AB7HESI,Appstore for Android,Scribblenauts Remix,3356.160,5520.0,0.99,Warner Bros,['THINK IT! CREATE IT! SOLVE IT! The award-win...,Software
2543144,868217,40099,0.8,B014RGFC0K,Appstore for Android,Smashy Road: Wanted,4343.616,7541.0,0.00,Bearbit Studios B.V.,['You are WANTED! Take your driving to new hei...,Software


### OHE DLA KATEGORII

In [3]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
print(df_user.tail())
# Check if columns exist before attempting operations
if "main_category" in df_user.columns and "category" in df_user.columns:
    cat_df = df_user[["main_category", "category"]].fillna("missing")
    
    ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    data = ohe.fit_transform(cat_df)
    
    df1 = pd.DataFrame(data, columns=ohe.get_feature_names_out(cat_df.columns), dtype=int)
    df_encoded = pd.concat([df_user.reset_index(drop=True), df1], axis=1)

    # Drop original categorical columns
    df_encoded = df_encoded.drop(["main_category", "category"], axis=1)
else:
    print("One or both of the required columns are missing.")
    df_encoded = df_user.copy()  # or handle accordingly
    
df_encoded.tail()

         user_id  item_id  rating parent_asin         main_category  \
2543141   868216    15203     1.0  B00B1TGUMG              Software   
2543142   868216    51839     1.0  B06XS4YRWX              Software   
2543143   868217    14052     0.8  B00AB7HESI  Appstore for Android   
2543144   868217    40099     0.8  B014RGFC0K  Appstore for Android   
2543145   868217    12895     1.0  B009HKL4B8  Appstore for Android   

                                                     title  average_rating  \
2543141  Microsoft Office Home and Student 2013 (1PC/1U...        1512.740   
2543142  NTI Echo 3, Make an exact copy of a HDD or SSD...          11.900   
2543143                                Scribblenauts Remix        3356.160   
2543144                                Smashy Road: Wanted        4343.616   
2543145                                  The Sims Freeplay       30263.040   

         rating_number  price                 store  \
2543141         1759.0    NaN             Microso

,user_id,item_id,rating,parent_asin,title,average_rating,rating_number,price,store,description,...,main_category_Computers,main_category_Gift Cards,main_category_Health & Personal Care,main_category_Home Audio & Theater,main_category_Premium Beauty,main_category_Software,main_category_missing,category_All_Beauty,category_Health_and_Personal_Care,category_Software
2543141,868216,15203,1.0,B00B1TGUMG,Microsoft Office Home and Student 2013 (1PC/1U...,1512.740,1759.0,NaN,Microsoft,['Product Description'\n 'Office Home & Studen...,...,0,0,0,0,0,1,0,0,0,1
2543142,868216,51839,1.0,B06XS4YRWX,"NTI Echo 3, Make an exact copy of a HDD or SSD...",11.900,17.0,NaN,Nti,"['NTI Echo'\n ""is the perfect solution for all...",...,0,0,0,0,0,1,0,0,0,1
2543143,868217,14052,0.8,B00AB7HESI,Scribblenauts Remix,3356.160,5520.0,0.99,Warner Bros,['THINK IT! CREATE IT! SOLVE IT! The award-win...,...,0,0,0,0,0,0,0,0,0,1
2543144,868217,40099,0.8,B014RGFC0K,Smashy Road: Wanted,4343.616,7541.0,0.00,Bearbit Studios B.V.,['You are WANTED! Take your driving to new hei...,...,0,0,0,0,0,0,0,0,0,1
2543145,868217,12895,1.0,B009HKL4B8,The Sims Freeplay,30263.040,42032.0,0.00,Electronic Arts,"['The Sims™ FreePlay lets you create, customiz...",...,0,0,0,0,0,0,0,0,0,1


## Dodajmy średnią dla ceny pomnożoną przez wagę

In [ ]:
df_cp = df_encoded 
for i in df_cp["user_id"].unique().tolist():
    df_tmp = df_user[df_user["user_id"] == int(i)]     
    avg_price = df_tmp["price"].sum() / df_tmp["price"].size
    df_cp[df_cp["price"] == i] = df_cp[df_cp["price"] == i].fillna(avg_price*df_tmp["rating"])
df_cp = df_cp.drop("rating_number", axis=1)
df_cp

KeyboardInterrupt: 

### Liczymy TF-IDF dla title, store, description

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(max_features=50000)

print(df_user.head())
## TWORZYMY MACIERZ TF-IDF
tf_title_matrix = vectorizer.fit_transform(df_user["title"].dropna())
tf_store_matrix = vectorizer.fit_transform(df_user["store"].fillna(""))
tf_description_matrix = vectorizer.fit_transform(df_user["description"].fillna(""))

### TWORZYMY USER MACIERZ TF-IDF
tf_user_title_matrix = vectorizer.fit_transform(df_user0["title"])
tf_user_store_matrix = vectorizer.fit_transform(df_user0["store"])
tf_user_description_matrix = vectorizer.fit_transform(df_user0["description"])

lsa = TruncatedSVD(n_components=100, algorithm='arpack')
lsa.fit(tf_title_matrix)
lsa.fit(tf_store_matrix)
lsa.fit(tf_description_matrix)

lsa.fit(tf_user_title_matrix)
lsa.fit(tf_user_store_matrix)
lsa.fit(tf_user_description_matrix)

### WYBIERAMY TYTUŁ 
# choosed_title = tf_title_matrix[0]

### PORÓWNUJEMY
cosine_sim1 = cosine_similarity(tf_user_title_matrix, tf_title_matrix)
cosine_sim2 = cosine_similarity(tf_user_store_matrix, tf_store_matrix)
cosine_sim3 = cosine_similarity(tf_user_description_matrix, tf_description_matrix)

similar_titles = list(enumerate(cosine_sim1[0]))
similar_stores = list(enumerate(cosine_sim2[0]))
similar_description = list(enumerate(cosine_sim3[0]))

p_titles = pd.DataFrame(similar_titles, columns=["product_id", "probability_titles"])
p_stores = pd.DataFrame(similar_stores, columns=["product_id", "probability_stores"])
p_description = pd.DataFrame(similar_description, columns=["product_id", "probability_description"])

df_probability = pd.merge(p_titles,  p_stores, on="product_id")
df_probability = pd.merge(df_probability, p_description, on="product_id")
df_probability["merged_prob"] = (df_probability["probability_titles"] * df_probability["probability_stores"] * df_probability["probability_description"])


def entropy(p):
    # Dodaj mały epsilon, żeby uniknąć log(0)
    p = np.clip(p, 1e-12, 1.0)
    return -p * np.log2(p)

df_probability["merged_entropy"] = entropy(df_probability["probability_titles"]) + entropy(df_probability["probability_stores"])


df_probability

   user_id  item_id  rating parent_asin         main_category  \
0        0     7314     1.0  B006CQ8TC2  Appstore for Android   
1        0    15493     1.0  B00B7S5FDG  Appstore for Android   
2        0    18817     0.8  B00DRPVE3M  Appstore for Android   
3        0    18251     0.8  B00D948SS0  Appstore for Android   
4        0    16199     0.2  B00BQPEIT2  Appstore for Android   

                                               title  average_rating  \
0                                      Turkey Season         212.800   
1  Another Monster at the End of This Book...Star...         395.600   
2  UNO ™ & Friends - The Classic Card Game Goes S...        7389.408   
3  World's Biggest Wordsearch - Your daily free w...        5040.320   
4                                     Temple Run: Oz         166.880   

   rating_number  price                    store  \
0          304.0   0.99        Gibbed Games, LLC   
1          460.0   3.99          Sesame Workshop   
2        11842.0    